In [1]:
import re, os
import unicodedata
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import nltk.sentiment

from wordcloud import WordCloud

from acquire_c import *
from prepare_c import *
from explore_c import *

plt.rc('figure', figsize=(13, 7))
plt.style.use('seaborn-darkgrid')

## Acquire data and find the dominant language in each row

In [3]:
# You can pass a threshold argument but the default is 75%
df = get_readme_data()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13792 entries, 0 to 13791
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   readme     13792 non-null  object
 1   prog_lang  13792 non-null  object
dtypes: object(2)
memory usage: 215.6+ KB


In [4]:
df.head()

,readme,prog_lang
0,🚧 WIP,JavaScript
1,abhisheknaiidu The CLI Table of Contents Ins...,JavaScript
2,Interview Preparation Arrays Sieve Catalan N...,C++
3,Hacktoberfest 2020 🎉 🗣 Hacktoberfest encour...,JavaScript
4,A-POP 🎶 A-POP is a HD Music Stream and Shar...,JavaScript


### We need to set a threshold to check for each programming language within each README row